# HW3, Movies
You will write you homework into this notebook.
You should probably create another notebook to test out your solutions.
After you are finished, make sure that your notebook can be run.
Run your notebook, and submit the results.
Do not delete the notebook cells that we supplied.
Add as many cells as you need.

<b>Fill in your name and netid in the following lines</b>

In [1]:
netid = "xxx"
name = "xxx yyy"
print netid
print name

xxx
xxx yyy


<b>The following lines import the modules we will use.
Do not import any others</b>

In [2]:
import pickle
import hashlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib notebook

## 1. Create a DataFrame by reading in movies.csv

In [3]:
moviedata=pd.read_csv("movies.csv")

## keep only the following columns:
title, id, rtAllCriticsRating, rtAllCriticsNumReviews

Feel free to give them more useful names.  Produce something like this.
It does not have to be exactly like this.  This is just an example.
Anything you can use to solve the rest will suffice.

In [4]:
moviedata=moviedata[["title","id","rtAllCriticsRating","rtAllCriticsNumReviews"]]

## 3. Make the ratings numeric.
Check the dtypes of your DataFrame.  
If the movie ratings are strings, force them to be numeric (say float64).
Remove all movies with null ratings and all movies that have no reviews.
Give evidence that you did this successfully.

Hint: look up the Pandas function `to_numeric`.  It gives one way of doing this.  There are others.
    

In [5]:
movieNum=moviedata
mytonumeric=lambda (x): pd.to_numeric(x,errors="coerce")

In [6]:
G=movieNum[["rtAllCriticsRating","rtAllCriticsNumReviews"]].apply(mytonumeric)

In [7]:
movieNum[["rtAllCriticsRating","rtAllCriticsNumReviews"]]=G
movieNumA=movieNum[movieNum.rtAllCriticsNumReviews>0]
movieNumA.shape

(9312, 4)

## Sanity check: you should now have 9312 movies in your table.
Show us that you do.

In [8]:
movieNumA.shape

(9312, 4)

## 4. Show us the 10 movies with the highest ratings, and the 10 with the lowest ratings.
We show the top and bottom 3.

In [9]:
movieNumA.nlargest(10,"rtAllCriticsRating")

,title,id,rtAllCriticsRating,rtAllCriticsNumReviews
6263,T�ky� monogatari,6643,9.6,34.0
7595,Our Hospitality,8609,9.5,17.0
8060,Orph�e,25954,9.5,15.0
842,Citizen Kane,923,9.4,54.0
6591,Ordet,6981,9.4,11.0
7525,Show Boat,8502,9.4,8.0
1138,Chinatown,1252,9.3,47.0
2502,Les quatre cents coups,2731,9.3,48.0
7987,Greed,25753,9.3,16.0
8080,Pickpocket,26052,9.3,19.0


In [10]:
movieNumA.nsmallest(10,"rtAllCriticsRating")

,title,id,rtAllCriticsRating,rtAllCriticsNumReviews
32,Wings of Courage,33,0.0,4.0
36,Across the Sea of Time,37,0.0,4.0
57,Le confessionnal,59,0.0,1.0
121,The Neverending Story III,126,0.0,3.0
122,Jupiter's Wife,128,0.0,4.0
123,Pie in the Sky,129,0.0,2.0
127,Sonic Outlaws,134,0.0,3.0
180,The Reckless Moment,189,0.0,3.0
233,Gumby: The Movie,244,0.0,4.0
270,Nobody's Fool,281,0.0,4.0


## 5. Create a DataFrame from `movie_actors.csv`.

In [11]:
movieactors=pd.read_csv("movie_actors.csv")

Apply head() to it.  You should get something like the following.
The movieID is the same as in our other file.
We will identify the actors by their actorIDs.
We will ignore the ranking, which is the order in which the actors were listed.

In [12]:
movieactors.head()

,movieID,actorID,actorName,ranking
0,1,annie_potts,Annie Potts,10
1,1,bill_farmer,Bill Farmer,20
2,1,don_rickles,Don Rickles,3
3,1,erik_von_detten,Erik von Detten,13
4,1,greg-berg,Greg Berg,17


## 6. Merge these two DataFrames together on movieID.
This will take a little work.
You should get something like the following.

In [13]:
movieactors=movieactors.rename(columns = {'movieID':'id'})

In [14]:
moviemerged=movieactors.merge(movieNumA,on="id")

## 7. The best actors.
For each actor, compute the average rating of the movies in which that actor appears.  View this as a rating of that actor.  Find the 10 actors with the highest ratings.  We give the top 3.

In [15]:
moviemerged.head()

,id,actorID,actorName,ranking,title,rtAllCriticsRating,rtAllCriticsNumReviews
0,1,annie_potts,Annie Potts,10,Toy story,9.0,73.0
1,1,bill_farmer,Bill Farmer,20,Toy story,9.0,73.0
2,1,don_rickles,Don Rickles,3,Toy story,9.0,73.0
3,1,erik_von_detten,Erik von Detten,13,Toy story,9.0,73.0
4,1,greg-berg,Greg Berg,17,Toy story,9.0,73.0


In [16]:
(moviemerged[["actorID","rtAllCriticsRating"]].groupby("actorID")).aggregate(np.mean).nlargest(3,"rtAllCriticsRating")

,rtAllCriticsRating
actorID,
mitsuhiro_mori,9.6
shiro_osaka,9.6
toru-abe,9.6


Some of these actors might look great just because they appeared in only one movie,
and it was good.  
We now restrict to actors that appeared in many movies.

## 8. Restricting to popular actors.
For each actor, find the number of movies in which that actor appeared.
Create a new DataFrame only contining actors who appeared in at least 37 movies.
This should give you 101 actors and 4500 rows.

Hint: one approach uses the Pandas method `isin` mentioned at the end of Lecture 6.
Another approach produces counts of movies per actor, like that below.

In [17]:
moviemerged.head()

,id,actorID,actorName,ranking,title,rtAllCriticsRating,rtAllCriticsNumReviews
0,1,annie_potts,Annie Potts,10,Toy story,9.0,73.0
1,1,bill_farmer,Bill Farmer,20,Toy story,9.0,73.0
2,1,don_rickles,Don Rickles,3,Toy story,9.0,73.0
3,1,erik_von_detten,Erik von Detten,13,Toy story,9.0,73.0
4,1,greg-berg,Greg Berg,17,Toy story,9.0,73.0


In [18]:
BIGACTORS=moviemerged.actorID.value_counts()[moviemerged.actorID.value_counts()>=37].index

In [19]:
moviebigactors=moviemerged[moviemerged['actorID'].isin(BIGACTORS)]

In [20]:
BIGMOVIES=moviebigactors.id.value_counts()[moviebigactors.id.value_counts()>=2].index
moviefiltered=moviebigactors[moviebigactors['id'].isin(BIGMOVIES)]
moviefiltered.shape

(2499, 7)

In [21]:
G=moviemerged.actorID.value_counts().to_frame()
G=G.rename(columns = {'actorID':'countmovies'})
indices=G.index
G["actorID"]=indices
G
moviemergewithcount=moviemerged.merge(G,on="actorID")

G=moviemerged.id.value_counts().to_frame()
G=G.rename(columns={'id':'movie_count'})
indices=G.index
G["id"]=indices
moviemergewithallcount=moviemergewithcount.merge(G,on="id")
moviemergewithallcount.head()

,id,actorID,actorName,ranking,title,rtAllCriticsRating,rtAllCriticsNumReviews,countmovies,movie_count
0,1,annie_potts,Annie Potts,10,Toy story,9.0,73.0,10,24
1,1,bill_farmer,Bill Farmer,20,Toy story,9.0,73.0,8,24
2,1,don_rickles,Don Rickles,3,Toy story,9.0,73.0,13,24
3,1,erik_von_detten,Erik von Detten,13,Toy story,9.0,73.0,11,24
4,1,greg-berg,Greg Berg,17,Toy story,9.0,73.0,4,24


In [22]:
moviedf=moviemergewithallcount[(moviemergewithallcount.countmovies>=37)&(moviemergewithallcount.movie_count>=37)]

## 9. Restricting the movies.

Now, create a DataFrame that only contains those popular actors and only those movies in which at least two of them appeared. This should leave you with 1032 movies and a DataFrame with 2499 lines.  This is the DataFrame we will use for the rest of this homework.

In [23]:
len(moviefiltered.id.unique())

1032

## 10. Confirm that each movie appears at least twice in this table.
Also compute the minimum number of times that an actor appears in this table.  We got 5.

In [25]:
print np.all(moviefiltered.id.value_counts()>=2),np.min(moviefiltered.actorID.value_counts())

True 5


## 11. Within this set find the 10 highest and lowest rated actors by giving them a rating equal to the average of the ratings of the movies in which they appear.

In [26]:
(moviefiltered[["actorID","rtAllCriticsRating"]].groupby("actorID")).aggregate(np.mean).nlargest(10,"rtAllCriticsRating")

,rtAllCriticsRating
actorID,
al_pacino,6.982609
jack_lemmon,6.973333
tom_hanks,6.810526
charlton_heston,6.788235
meryl_streep,6.719231
robert_duvall,6.708824
john_wayne,6.700000
clint_eastwood,6.690476
robert_de_niro,6.587234


In [27]:
(moviefiltered[["actorID","rtAllCriticsRating"]].groupby("actorID")).aggregate(np.mean).nsmallest(10,"rtAllCriticsRating")

,rtAllCriticsRating
actorID,
keith_david,4.730769
dan_aykroyd,4.810345
frank_welker,4.842105
rip_torn,4.973684
bruce_willis,4.995833
whoopi_goldberg,5.204545
max_von_sydow,5.256250
john_cleese,5.333333
marshall_bell,5.342308


# Rating by Least Squares
The drawback of the above rating scheme is that it does not distinguish each actor's 
individual contribution to a movie.  We would like to do that by assigning a rating to each actor so that the rating of every movie is the average of the ratings of actors in it.

We can't do that exactly, but we can try to come close with least squares.


# 12. Building the matrix
Construct a matrix whose rows are indexed by the movies and whose columns are indexed by the actors in the last DataFrame.  I don't mean "indexed" in a formal sense.  Rather, it should have 101 columns and 1032 rows.  It should have a nonzero in row r and column c when actor c appears in movie r.  In this case, the entry should be the the reciprocal of the number of actors in that movie.  This way, multiplying by a vector encoding the ratings of actors will compute the average rating of the actors in each movie.

Hint: it is possible to do this using either DataFrame operations or Dictionaries.  Some might find the Pandas `iterrows` or `itertuples` method, or the Python function `zip`, to be useful.

If you case you cannot construct this matrix, we will create a file that you can use to include it.  This will allow you to complete the rest of the problem set (at some loss of points).

In [ ]:
actors=moviefiltered.actorID.unique()
movies=moviefiltered.id.unique()
a=pd.DataFrame(data=0.,index=movies,columns=actors)
L=moviefiltered.groupby(by="id",as_index=False)
V={j[0]:j[1].shape[0] for j in L}

In [ ]:
for j in moviefiltered.itertuples():
    t_movieid=j.id
    t_actorid=j.actorID
    a[t_actorid][t_movieid]=1.0/(V[t_movieid])

In [ ]:
moviefiltered.head()
mydict={}
for j in moviefiltered.itertuples():
    if j.id in mydict:
        mydict[j.id].append(j.actorID)
    else:
        mydict[j.id]=[j.actorID]

Adict=dict.fromkeys(moviefiltered.id.unique())
v=dict.fromkeys(moviefiltered.actorID.unique(),0)
for j in moviefiltered.itertuples():
    if Adict[j.id]:
        Adict[j.id][j.actorID]=1.0/len(mydict[j.id])
    else:
        Adict[j.id]=v.copy()
        Adict[j.id][j.actorID]=1.0/len(mydict[j.id])

bigmat=pd.DataFrame.from_dict(Adict,orient='index')

In [ ]:
bigmat.shape

In [ ]:
np.sum(np.isclose(a.sum(axis=1),1))

## 13. Make a vector of the movie ratings, indexed in the same way as the columns of the matrix.

In [ ]:
moviefiltered.head()
A=bigmat

In [ ]:
GG=moviefiltered[["id","rtAllCriticsRating"]].groupby(by="id",as_index=False)
b=GG.aggregate(np.mean).set_index("id")
b.head()

## 14. Solve the least squares problem to find the ratings for actors.
Then, compute the mean of the squares of the entries of the vector
$$ Ax - b$$.
To test how good this is, also compute the mean of the squares of
b minus the mean of b.

In [ ]:
xhat=np.linalg.solve(A.T.dot(A),A.T.dot(b))
VA=A.dot(xhat)
VA.columns=["rtAllCriticsRating"]

np.linalg.norm(VA-b)**2/A.shape[0]

In [ ]:
np.linalg.norm(b.mean()-b)**2/A.shape[0]

## 15. According to these ratings, whose are the 10 best and worst actors (out of the 101 we are considering)?

# Should we believe these top (and bottom) 10 lists?
We will divide the movies into a training set and a test set.
We will use the training set to assign ratings to actors, and then see how well they explain the ratings of movies in the test set.
We will use your netid to determine which movies go in which set.

## 16. Map movies to train and test.
Create a pandas Series or a list or an array containing the movie ids.
The following function assigns True or False to a movie id.
Apply it to all of the movie ids in the 101 we are considering.
Verify that the number of True values is about 87.5%.
You can do this by computing the sum of a boolean vector.

In [ ]:
# DO NOT CHANGE THIS CODE!
def is_train(movie_id):
    hash_object = hashlib.md5(netid + str(movie_id))
    hex_hash = hash_object.hexdigest()
    return int(hex_hash[0],16) < 14   
is_train('1')

In [ ]:
A.index

In [ ]:
trainingindex=([is_train(x) for x in A.index])

## 17. Compute the Least Squares Solution on the trainind data.
Then, report the top and bottom 10 actors under this new rating vector.

In [ ]:
Atrain=A[trainingindex]
btrain=b[trainingindex]
xhat=np.linalg.solve(Atrain.T.dot(Atrain)+np.identity(101)*0.1,Atrain.T.dot(btrain))
VA=Atrain.dot(xhat)
VA.columns=["rtAllCriticsRating"]

np.linalg.norm(VA-btrain)**2/Atrain.shape[0]

## 17. Evaluate how the least squares solution on train performs on test.
This time, compute the mean squared error on the ratings of movies in the test set, and compare it to the mean squared error if we just guessed the average rating for those movies.

In [ ]:
VA=A[np.logical_not(trainingindex)].dot(xhat)
VA.columns=["rtAllCriticsRating"]
np.linalg.norm(VA-b[np.logical_not(trainingindex)])**2/np.sum(np.logical_not(trainingindex))

In [ ]:
np.linalg.norm(np.mean(b[np.logical_not(trainingindex)])-b[np.logical_not(trainingindex)])**2/np.sum(np.logical_not(trainingindex))

## 18. How stable were your top 10 and bottom 10?
Give any observations you can make about the difference between dividing into train and test, 
and when we run on the whole data.  That is, what do you think of these actor ratings that we created?  Give your answer in a Markdown cell.